In [18]:
# !pip install transformers
# !pip install -q torch torchvision torchaudio
# !pip install supabase
# !pip install dotenv
# !pip install finnhub-python

In [ ]:
from google.colab import files,drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/gdrive/MyDrive/finn-project/be-ai-model')

from source.input_processing import get_csv_data, merge_data
from source.sentiment_model import analyze_sentiment_with_progress, add_integer_column, sums_sentiment_score_for_7_days, update_sentiment_score_in_db
from source.lstm_model import get_scale_data, get_scale_data_with_fit, create_sequences_for_train, compile_model, train_model, predict_prices, create_sequences_for_prod
from source.output_processing import compare_prices_with_graph
from source.market_capitalization import get_capitalization

In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from supabase import create_client, Client
import finnhub
from dotenv import load_dotenv

dotenv_path = 'gdrive/MyDrive/finn-project/.env'
load_dotenv(dotenv_path=dotenv_path)

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_API_KEY")
finnhub_api_key = os.environ.get("FINNHUB_API_KEY")

SEQUENCE_LENGTH = 7
SENTIMENT_WINDOW_DAYS = 7
FETCH_DAYS = 13

supabase: Client = create_client(supabase_url, supabase_key)
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

In [20]:
# TSLA에 관한 모델만 1개만 생성되어있으므로, db에서 테슬라에 관한 주가/뉴스 데이터만 가져온다.
stock_response = supabase.table('stocks').select('id,stock_code,company_name').eq('stock_code', 'TSLA').single().execute()
stock_id = stock_response.data['id']
stock_code = stock_response.data['stock_code']
company_name = stock_response.data['company_name']

In [5]:
def get_price_data_from_db(stock_id):
  prices_latest_date_response = supabase.table('stock_prices').select('price_date') \
          .eq('stock_id', stock_id) \
          .order('price_date', desc=True) \
          .limit(1) \
          .single() \
          .execute()

  if not prices_latest_date_response.data:
      print(f"🚨 '{stock_id}'에 대한 최근의 주가 데이터가 DB에 없습니다.")

  prices_latest_date = prices_latest_date_response.data['price_date']

  prices_response = supabase.table('stock_prices').select('*') \
          .eq('stock_id', stock_id) \
          .lte('price_date', prices_latest_date) \
          .order('price_date', desc=True) \
          .limit(FETCH_DAYS) \
          .execute()

  return prices_response.data

In [6]:
def get_news_data_from_db(start_date, end_date, stock_id):

  news_response = supabase.table('news').select('*').eq('stock_id', stock_id) \
  .gte('created_date', start_date.strftime('%Y-%m-%d')) \
  .lte('created_date', end_date.strftime('%Y-%m-%d')) \
  .execute()

  if not news_response.data:
      print(f"🚨 '{stock_id}'에 대한 최근의 뉴스 데이터가 DB에 없습니다.")

  return news_response.data

In [ ]:
from tensorflow.keras.models import load_model
import pickle

def get_existing_model():
  load_path = '/gdrive/MyDrive/finn-project/be-ai-model/models/tsla_finn_model.keras'
  return load_model(load_path)

def get_existing_scaler():
  load_path = '/gdrive/MyDrive/finn-project/be-ai-model/models/tsla_finn_scaler.pkl'
  with open(load_path, 'rb') as f:
      scaler = pickle.load(f)
  return scaler

In [75]:
def get_change_rate(prev_price, today_price):
    change_rate = ((today_price - prev_price) / prev_price) * 100
    return change_rate.round(2)

def get_closely_prev_close_price(df):
    # 2. 'close_price'가 NaN(비어있지 않은)이 아닌 행만 필터링합니다.
    valid_data_df = df.dropna(subset=['close_price'])

    # 3. 인덱스(date)를 기준으로 내림차순 정렬하여 가장 최신 데이터가 맨 위로 오게 합니다.
    sorted_df = valid_data_df.sort_index(ascending=False)

    latest_valid_row = sorted_df.iloc[0]
    
    # 인덱스가 날짜이므로, .name 속성으로 날짜를 가져옵니다.
    latest_date = latest_valid_row.name.strftime('%Y-%m-%d')
    latest_close_price = latest_valid_row['close_price']

    
    return latest_close_price



In [83]:
# Predictions row를 만들고, db에 저장(시가총액 정보도 호출하여 저장)
def save_predictions_in_db(stock_id, stock_code, company_name, prediction_price, prediction_date, change_rate, capitalization):
    try :
        response = supabase.table('predictions') \
        .upsert({"stock_id" : stock_id, "prediction_date": prediction_date, "stock_code" : stock_code,
                "company_name" : company_name, "prediction_price" : prediction_price, "change_rate" : change_rate,
                "capitalization" : capitalization}) \
        .execute()
        
        if hasattr(response, 'error') and response.error is not None:
                print(f"🚨 DB 업데이트 중 에러가 발생했습니다: {response.error}")
        else:
            print("✅ DB 업데이트가 성공적으로 완료되었습니다.")
    except Exception as e:
        print(f"🚨 DB 업데이트 중 예외 발생: {e}")

In [7]:
prices_response = get_price_data_from_db(stock_id)
stock_prices_df = pd.DataFrame(prices_response)
stock_prices_df = stock_prices_df.rename(columns={'price_date':'date', 'id' : 'stock_price_id'})
start_date = pd.to_datetime(stock_prices_df['date']).min()
end_date = pd.to_datetime(stock_prices_df['date']).max()

news_response = get_news_data_from_db(start_date, end_date, stock_id)
news_df = pd.DataFrame(news_response)
news_df = news_df.rename(columns={'created_date':'date', 'id' : 'news_id'})

In [10]:
# 모델 로드
model = get_existing_model()
scaler = get_existing_scaler()

2025-06-12 16:34:37.221192: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-06-12 16:34:37.221230: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-06-12 16:34:37.221235: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-06-12 16:34:37.221280: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-12 16:34:37.221294: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/conda/Downloads/finn-project/be-ai-model/.venv/lib/python3.11/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from versio

In [11]:
# 감정평가 수행 후, sentiment_score를 db news 테이블에 새롭게 업데이트한다.
news_df = analyze_sentiment_with_progress(news_df)

감성 분석 진행중:   0%|          | 0/15 [00:00<?, ?batch/s]

In [58]:
merged_df = merge_data(news_df, stock_prices_df)

In [59]:
merged_df['sentiment_influence'] = 0.0
merged_df = add_integer_column(merged_df)

sums_sentiment_score_for_7_days(merged_df)

  0%|          | 0/464 [00:00<?, ?it/s]

In [14]:
update_sentiment_score_in_db(supabase, merged_df)

🔄 464개의 감성 점수를 DB에 업데이트합니다...
✅ DB 업데이트가 성공적으로 완료되었습니다.


In [61]:
merged_df

,news_id,title,sentiment,confidence,stock_price_id,change_rate,close_price,high_price,low_price,open_price,stock_id,volume,adj_close_price,sentiment_influence,sentiment_score
date,,,,,,,,,,,,,,,
2025-05-23,31bdf9d9-847d-4bfb-9a7a-db97f88b61ab,Robotaxi Fever Fuels $500 Tesla Stock Price Ta...,positive,0.6773,1fe120e2-5648-4fb7-9908-02ac4927091c,0.0,339.34,343.18,333.21,337.920,c695fc5b-eb68-4fc9-ab14-a16b24af6b37,84654818.0,339.34,-0.693800,1
2025-05-24,79b2bbdf-21d6-4a5a-969a-7eba9732d7a0,2025.14.100.1 Official Tesla Release Notes - S...,neutral,0.9483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.270030,0
2025-06-01,7b674e8c-6e7f-4827-8072-0c9c340ad72a,Chinese EV Makers Pull Away From Tesla With Sa...,negative,0.7068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.730640,-1
2025-06-01,9289ac42-2b0c-411f-b5d2-40d6e43ed771,Tesla Model 3’s Reign Challenged: These Long-R...,positive,0.7283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.730640,1
2025-06-02,c5ba1739-475b-466a-88cd-830da9589186,Tesla's Affordable EV Denials Sent The Company...,negative,0.8362,da4d93a7-6533-499e-86f0-0cbe6e5ebebf,0.0,342.69,348.02,333.33,343.500,c695fc5b-eb68-4fc9-ab14-a16b24af6b37,81873829.0,342.69,-8.973740,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,0e533e3a-2e7a-4f93-b11e-281fde368e8b,Tesla Stock Ekes Out a 4th Straight Day of Gai...,positive,0.7113,87067fe9-1e51-403d-bd7a-bef25a3881f2,0.0,326.43,335.50,322.50,334.395,c695fc5b-eb68-4fc9-ab14-a16b24af6b37,122611360.0,326.43,-21.767525,1
2025-06-11,9ee7b58f-3698-447c-90c5-5b3fe009c3bb,Elon Musk Sets Tentative Tesla Robotaxi Launch...,neutral,0.9298,87067fe9-1e51-403d-bd7a-bef25a3881f2,0.0,326.43,335.50,322.50,334.395,c695fc5b-eb68-4fc9-ab14-a16b24af6b37,122611360.0,326.43,-21.767525,0
2025-06-11,b5da94d3-b10e-4868-a0c6-c9281bd3c0ea,The Latest Tempest for Tesla's Stock Looks to ...,neutral,0.5397,87067fe9-1e51-403d-bd7a-bef25a3881f2,0.0,326.43,335.50,322.50,334.395,c695fc5b-eb68-4fc9-ab14-a16b24af6b37,122611360.0,326.43,-21.767525,0


In [48]:
features = ['sentiment_influence', 'open', 'high', 'low', 'adjClose', 'volume']
target   = 'close'
all_cols = features + [target]

dropped_merged_df = merged_df.rename(columns={'open_price':'open', 'high_price' : 'high', 'low_price' : 'low', 'close_price' : 'close', 'adj_close_price' : 'adjClose'})
dropped_merged_df = dropped_merged_df[ features + [target] ].dropna()

scaled = get_scale_data(scaler, dropped_merged_df)
X = create_sequences_for_prod(scaled)

In [44]:
y_pred_scaled = predict_prices(model, X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [45]:
all_cols = ['sentiment_influence', 'open', 'high', 'low', 'adjClose', 'volume', 'close']
features = all_cols[:-1] # 'close'를 제외한 모든 컬럼
target = 'close'
target_col_index = all_cols.index(target)

num_features = len(features)
dummy_array = np.zeros((len(y_pred_scaled), len(all_cols)))
# 'close' 위치(6번 인덱스)에 예측된 값을 삽입
dummy_array[:, target_col_index] = y_pred_scaled.ravel()
# Scaler를 이용해 전체 배열을 역변환하고, 'close' 컬럼만 추출
y_pred_actual = scaler.inverse_transform(dummy_array)[:, target_col_index]

In [84]:
next_day_predicted_close = y_pred_actual[-1].round(4)
closely_prev_price = get_closely_prev_close_price(merged_df)
change_rate = get_change_rate(closely_prev_price, next_day_predicted_close)
print(f"예측된 실제 종가: ${next_day_predicted_close:.4f}")
capitalization = get_capitalization(finnhub_client, stock_code)
print(capitalization)

예측된 실제 종가: $325.1013
1049967


In [86]:
today_date = datetime.now().strftime("%Y-%m-%d")
save_predictions_in_db(stock_id, stock_code, company_name, next_day_predicted_close, today_date, change_rate, capitalization)

✅ DB 업데이트가 성공적으로 완료되었습니다.
